In [ ]:
import mysql.connector
from mysql.connector import Error


try:
    # Connect to MySQL database
    connection = mysql.connector.connect(
        host='localhost',           # your host
        user='root',       # your username
        password='root',   # your password
        database='ecom_data',   # your database name
        allow_local_infile=True     # enable local infile loading
    )

    if connection.is_connected():
        cursor = connection.cursor()

        # Path to your CSV file (change as required)
        file_path = r'C:\ProgramData\MySQL\MySQL Server 8.0\Uploads\dim_customers.csv'

        # SQL query to load data from file into the table
        load_sql = f"""
        LOAD DATA LOCAL INFILE '{file_path}'
        INTO TABLE dim_customers
        FIELDS TERMINATED BY ',' 
        ENCLOSED BY '"'
        LINES TERMINATED BY '\\n'
        IGNORE 1 LINES;
        """

        cursor.execute(load_sql)
        connection.commit()
        print("Data loaded successfully from file")

except Error as e:
    print("Error:", e)

finally:
    if 'connection' in locals() and connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection closed")




In [ ]:
import mysql.connector
from mysql.connector import Error
import traceback

try:
    connection = mysql.connector.connect(
        host='localhost',
        user='root',
        password='root',
        database='ecom_data',
        allow_local_infile=True
    )
    if connection.is_connected():
        cursor = connection.cursor()
        file_path = r'C:\ProgramData\MySQL\MySQL Server 8.0\Uploads\dim_customers.csv'
        load_sql = f"""
        LOAD DATA LOCAL INFILE '{file_path}'
        INTO TABLE dim_customers
        FIELDS TERMINATED BY ',' 
        ENCLOSED BY '"'
        LINES TERMINATED BY '\\n'
        IGNORE 1 LINES;
        """
        cursor.execute(load_sql)
        connection.commit()
        print("Data loaded successfully from file")

except Error as e:
    print("MySQL error:", e)
    traceback.print_exc()

except Exception as e:
    print("Unexpected error:", e)
    traceback.print_exc()

finally:
    if 'connection' in locals() and connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection closed")


In [2]:
pip install caas_jupyter_tools


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement caas_jupyter_tools (from versions: none)
ERROR: No matching distribution found for caas_jupyter_tools


In [1]:
# Synthetic Olist-style dataset generator (2018-09 to 2023-12)
# This script creates CSVs matching the Olist schema with realistic macro effects (COVID-19, supply chain, inflation, recovery).
# Files will be saved under /mnt/data/olist_synthetic/ and zipped for easy download.

import os, io, zipfile, math, random, string, json
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
from caas_jupyter_tools import display_dataframe_to_user

random.seed(42)
np.random.seed(42)

OUT_DIR = r"D:\Portfolio Projects\Ecommerce\Data\olist_synthetic"
os.makedirs(OUT_DIR, exist_ok=True)

# -----------------------------
# 1) CONFIG
# -----------------------------

start_date = pd.Timestamp("2018-09-01")
end_date   = pd.Timestamp("2023-12-31")

# Regions & states of Brazil (abridged but broad coverage)
states = [
    ("SP", "Southeast"), ("RJ", "Southeast"), ("MG", "Southeast"), ("ES", "Southeast"),
    ("PR", "South"), ("SC", "South"), ("RS", "South"),
    ("BA", "Northeast"), ("PE", "Northeast"), ("CE", "Northeast"), ("RN", "Northeast"), ("PB", "Northeast"), ("AL", "Northeast"), ("MA", "Northeast"),
    ("DF", "Central-West"), ("GO", "Central-West"), ("MT", "Central-West"), ("MS", "Central-West"),
    ("PA", "North"), ("AM", "North"), ("RO", "North"), ("RR", "North"), ("AP", "North"), ("AC", "North"), ("TO", "North"),
    ("PI", "Northeast"), ("SE", "Northeast")
]

# Some Brazilian city names (sample; synthetic)
cities_by_state = {
    "SP": ["São Paulo","Campinas","Santos","São José dos Campos","Ribeirão Preto","Sorocaba"],
    "RJ": ["Rio de Janeiro","Niterói","Duque de Caxias","Nova Iguaçu","Petrópolis"],
    "MG": ["Belo Horizonte","Uberlândia","Contagem","Juiz de Fora","Uberaba"],
    "ES": ["Vitória","Vila Velha","Serra","Cariacica"],
    "PR": ["Curitiba","Londrina","Maringá","Foz do Iguaçu"],
    "SC": ["Florianópolis","Joinville","Blumenau","Itajaí"],
    "RS": ["Porto Alegre","Caxias do Sul","Pelotas","Canoas"],
    "BA": ["Salvador","Feira de Santana","Vitória da Conquista","Ilhéus"],
    "PE": ["Recife","Olinda","Caruaru","Petrolina"],
    "CE": ["Fortaleza","Caucaia","Juazeiro do Norte","Sobral"],
    "RN": ["Natal","Mossoró","Parnamirim"],
    "PB": ["João Pessoa","Campina Grande","Patos"],
    "AL": ["Maceió","Arapiraca","Palmeira dos Índios"],
    "MA": ["São Luís","Imperatriz","Caxias"],
    "DF": ["Brasília"],
    "GO": ["Goiânia","Aparecida de Goiânia","Anápolis"],
    "MT": ["Cuiabá","Várzea Grande","Rondonópolis"],
    "MS": ["Campo Grande","Dourados","Três Lagoas"],
    "PA": ["Belém","Ananindeua","Santarém"],
    "AM": ["Manaus","Parintins","Itacoatiara"],
    "RO": ["Porto Velho","Ji-Paraná"],
    "RR": ["Boa Vista"],
    "AP": ["Macapá"],
    "AC": ["Rio Branco"],
    "TO": ["Palmas","Araguaína"],
    "PI": ["Teresina","Parnaíba"],
    "SE": ["Aracaju","Nossa Senhora do Socorro"]
}

# Product categories (subset of original Olist; add some)
categories = [
    ("bed_bath_table", 80, 600),
    ("health_beauty", 30, 700),
    ("sports_leisure", 50, 1000),
    ("computers_accessories", 100, 2500),
    ("furniture_decor", 120, 2000),
    ("housewares", 40, 900),
    ("watches_gifts", 90, 1800),
    ("telephony", 120, 3000),
    ("garden_tools", 40, 900),
    ("auto", 70, 1500),
    ("toys", 30, 600),
    ("pet_shop", 15, 400),
    ("baby", 20, 500),
    ("stationery", 10, 300),
    ("fashion_bags_accessories", 20, 700),
    ("cool_stuff", 15, 800),
    ("luggage_accessories", 60, 1200),
    ("kitchen_dining_laundry_garden_furniture", 70, 2200),
]

# Category name translations (simplified)
translations = {c[0]: c[0].replace("_"," ") for c in categories}

# Macro scenario multipliers by month (affect orders, delivery delays, cancellations, reviews, prices)
# Base monthly order demand (relative)
date_range = pd.date_range(start_date, end_date, freq="D")
monthly_index = pd.Series(1.0, index=pd.period_range(start=start_date, end=end_date, freq="M"))

def set_period_multiplier(start, end, factor):
    p = pd.period_range(start, end, freq="M")
    for m in p:
        monthly_index.loc[m] *= factor

# Baseline: gentle growth 2019–2023
for m in monthly_index.index:
    year = m.start_time.year
    if year == 2019:
        monthly_index.loc[m] *= 1.05
    elif year == 2020:
        monthly_index.loc[m] *= 1.10  # more online shopping overall
    elif year == 2021:
        monthly_index.loc[m] *= 1.15
    elif year == 2022:
        monthly_index.loc[m] *= 1.12
    elif year == 2023:
        monthly_index.loc[m] *= 1.18

# COVID shock: 2020-03 to 2020-05 dip (supply/logistics issues), then surge 2020-06..2020-12
set_period_multiplier("2020-03", "2020-05", 0.80)
set_period_multiplier("2020-06", "2020-12", 1.15)

# 2021 supply-chain constraints: demand high but fulfillment slower (we'll model delays elsewhere)
set_period_multiplier("2021-01", "2021-08", 1.05)

# 2022 inflation cools demand a bit
set_period_multiplier("2022-01", "2022-12", 0.97)

# Seasonality: holiday peaks (Black Friday/Cyber Week + Christmas)
for m in monthly_index.index:
    if m.start_time.month == 11:
        monthly_index.loc[m] *= 1.30
    if m.start_time.month == 12:
        monthly_index.loc[m] *= 1.25

# Carnival (Feb) modest lift for certain categories
for m in monthly_index.index:
    if m.start_time.month == 2:
        monthly_index.loc[m] *= 1.05

# Price inflation factors by year
inflation_by_year = {2018: 1.00, 2019: 1.04, 2020: 1.06, 2021: 1.12, 2022: 1.20, 2023: 1.25}

# Logistics delay baseline (days) per year (COVID/supply chain)
delay_year_base = {2018: 10, 2019: 9, 2020: 13, 2021: 14, 2022: 11, 2023: 9}

# Cancellation rate (portion of orders) by year
cancel_rate_year = {2018: 0.03, 2019: 0.025, 2020: 0.04, 2021: 0.035, 2022: 0.03, 2023: 0.025}

# Review score mean shift by year (lower during 2020–2021)
review_mean_by_year = {2018: 4.1, 2019: 4.2, 2020: 3.9, 2021: 3.95, 2022: 4.05, 2023: 4.15}

# -----------------------------
# 2) Helper Generators
# -----------------------------

def rand_id(prefix, n=32):
    return prefix + "_" + "".join(random.choices(string.ascii_lowercase + string.digits, k=n))

def pick_state_city():
    st, region = random.choice(states)
    city = random.choice(cities_by_state[st])
    return st, city

def zip_prefix():
    return int(random.uniform(10000, 99999))

def geo_coord_for_state(st):
    # rough bounding boxes (synthetic)
    boxes = {
        "SP": (-24.0, -45.0, -20.0, -47.0),
        "RJ": (-23.4, -43.7, -21.5, -41.0),
        "MG": (-20.0, -48.5, -14.0, -40.0),
        "RS": (-33.8, -57.6, -27.0, -49.7),
        "SC": (-28.5, -53.7, -26.0, -48.4),
        "PR": (-26.3, -54.6, -22.5, -48.0),
        "BA": (-18.0, -46.0, -8.0, -38.0),
        "PE": (-9.6, -38.0, -7.5, -34.8),
        "CE": (-7.4, -41.4, -2.8, -37.0),
        "PA": (-5.0, -55.0, 1.0, -47.0),
        "AM": (-7.0, -63.0, 0.0, -59.0),
        "ES": (-21.3, -41.2, -18.0, -39.0),
        "GO": (-18.5, -52.5, -15.0, -48.5),
        "DF": (-16.1, -48.2, -15.6, -47.7),
        "MT": (-16.7, -58.7, -12.5, -54.7),
        "MS": (-22.6, -56.9, -19.6, -53.2),
        "RO": (-13.9, -65.0, -8.5, -60.5),
        "RR": (1.5, -61.5, 4.5, -59.5),
        "AP": (0.0, -52.1, 2.7, -50.6),
        "AC": (-11.0, -70.0, -7.0, -67.0),
        "TO": (-12.5, -50.5, -8.0, -45.5),
        "RN": (-6.4, -36.9, -5.1, -34.8),
        "PB": (-7.4, -37.7, -6.0, -34.8),
        "AL": (-10.8, -38.3, -9.2, -35.0),
        "MA": (-5.7, -47.5, -1.3, -42.8),
        "PI": (-9.4, -46.5, -2.9, -41.0),
        "SE": (-11.6, -38.2, -10.0, -36.4),
    }
    lat1, lng1, lat2, lng2 = boxes.get(st, (-23.0, -47.0, -22.0, -46.0))
    lat = random.uniform(min(lat1, lat2), max(lat1, lat2))
    lng = random.uniform(min(lng1, lng2), max(lng1, lng2))
    return lat, lng

# -----------------------------
# 3) Create dimension tables: products, sellers, customers, geolocation
# -----------------------------

N_PRODUCTS = 6000
N_SELLERS = 2500
N_CUSTOMERS = 120000
N_ZIPCODES = 15000

# Products
product_ids = [rand_id("prod") for _ in range(N_PRODUCTS)]
product_rows = []
for pid in product_ids:
    cat, minp, maxp = random.choice(categories)
    name_len = np.random.randint(20, 60)
    desc_len = np.random.randint(200, 1200)
    photos = np.random.randint(1, 5)
    weight = int(np.random.gamma(shape=4, scale=300) + 100)
    length = int(np.random.normal(30, 10))
    height = int(np.random.normal(15, 6))
    width  = int(np.random.normal(25, 8))
    product_rows.append([pid, cat, name_len, desc_len, photos, weight, length, height, width])

products_df = pd.DataFrame(product_rows, columns=[
    "product_id","product_category_name","product_name_lenght","product_description_lenght",
    "product_photos_qty","product_weight_g","product_length_cm","product_height_cm","product_width_cm"
])

# Sellers
seller_ids = [rand_id("sell") for _ in range(N_SELLERS)]
seller_rows = []
for sid in seller_ids:
    st, city = pick_state_city()
    seller_rows.append([sid, zip_prefix(), city, st])
sellers_df = pd.DataFrame(seller_rows, columns=["seller_id","seller_zip_code_prefix","seller_city","seller_state"])

# Customers
customer_ids = [rand_id("cust") for _ in range(N_CUSTOMERS)]
unique_ids = [rand_id("uniq", 16) for _ in range(N_CUSTOMERS)]
customer_rows = []
for cid, uid in zip(customer_ids, unique_ids):
    st, city = pick_state_city()
    customer_rows.append([cid, uid, zip_prefix(), city, st])
customers_df = pd.DataFrame(customer_rows, columns=[
    "customer_id","customer_unique_id","customer_zip_code_prefix","customer_city","customer_state"
])

# Geolocations (zip prefix to coordinates)
zips = random.sample(range(10000, 99999), N_ZIPCODES)
geo_rows = []
for z in zips:
    st, city = pick_state_city()
    lat, lng = geo_coord_for_state(st)
    geo_rows.append([z, lat, lng, city, st])
geolocation_df = pd.DataFrame(geo_rows, columns=[
    "geolocation_zip_code_prefix","geolocation_lat","geolocation_lng","geolocation_city","geolocation_state"
])

# Category translations
trans_rows = [[c[0], translations[c[0]]] for c in categories]
translation_df = pd.DataFrame(trans_rows, columns=["product_category_name","product_category_name_english"])

# -----------------------------
# 4) Generate orders over time with macro effects
# -----------------------------

# Daily base orders starting at ~120 and scaled by month index + noise
daily_dates = pd.date_range(start_date, end_date, freq="D")
daily_orders = []
base_daily = 120

for d in daily_dates:
    month_period = d.to_period("M")
    seasonal = 1.0
    # weekend boost for online shopping
    if d.weekday() in (5, 6):
        seasonal *= 1.08
    # micro events: prime-like mid-July, hot sales
    if d.month == 7 and d.day in range(10,18):
        seasonal *= 1.12
    # Black Friday week (last Fri of Nov) -> big spike
    if d.month == 11 and d.weekday() == 4 and 23 <= d.day <= 29:
        seasonal *= 1.6
    # Christmas run-up
    if d.month == 12 and d.day >= 10:
        seasonal *= 1.25
    
    mu = base_daily * monthly_index.loc[month_period] * seasonal
    # Poisson-like variability
    count = max(0, int(np.random.normal(mu, mu**0.5)))
    daily_orders.append((d, count))

orders_per_day = dict(daily_orders)
TOTAL_ORDERS = sum(count for _, count in daily_orders)

# -----------------------------
# 5) Build Orders, Items, Payments, Reviews
# -----------------------------

order_rows = []
item_rows  = []
payment_rows = []
review_rows = []

def sample_product():
    row = products_df.sample(1).iloc[0]
    return row["product_id"], row["product_category_name"]

def price_for_category(cat, date):
    base = [c for c in categories if c[0]==cat][0]
    minp, maxp = base[1], base[2]
    year = date.year
    infl = inflation_by_year.get(year, 1.0)
    p = np.random.uniform(minp, maxp) * infl
    return round(float(p), 2)

def freight_from_price(price):
    # freight roughly 8-15% of price
    return round(float(price * np.random.uniform(0.08, 0.15)), 2)

def delivery_dates(purchase_dt):
    y = purchase_dt.year
    base = delay_year_base.get(y, 10)
    noise = max(0, int(np.random.normal(0, 3)))
    delivered_days = base + noise
    est_days = max(2, int(np.random.normal(base-1, 2)))
    approved = purchase_dt + pd.to_timedelta(np.random.randint(0, 48), unit="h")
    ship = approved + pd.to_timedelta(np.random.randint(12, 72), unit="h")
    delivered = ship + pd.to_timedelta(delivered_days, unit="D")
    est = purchase_dt + pd.to_timedelta(est_days, unit="D")
    return approved, ship, delivered, est

def order_status_for_year(y):
    r = random.random()
    cancel = cancel_rate_year.get(y, 0.03)
    if r < cancel:
        return "canceled"
    # a tiny portion "shipped" or "invoiced"
    if r < cancel + 0.03:
        return random.choice(["shipped","invoiced"])
    return "delivered"

def review_score_for_year(y, delayed=False):
    mean = review_mean_by_year.get(y, 4.0)
    sd = 0.8
    score = int(np.clip(round(np.random.normal(mean - (0.4 if delayed else 0), sd)), 1, 5))
    return score

# For mapping: pick random customers/sellers weighted by state populations (simple: SP more weight)
state_weight = {st: (3.0 if st=="SP" else 1.0) for st,_ in states}
seller_state = sellers_df["seller_state"].map(state_weight).values
seller_probs = seller_state / seller_state.sum()

customer_state = customers_df["customer_state"].map(state_weight).values
customer_probs = customer_state / customer_state.sum()

order_id_seq = 0
review_id_seq = 0

for d, count in daily_orders:
    for _ in range(count):
        order_id = rand_id("ord", 20)
        # link a customer and pick 1–4 items, possibly multiple sellers
        cust_idx = np.random.choice(len(customers_df), p=customer_probs)
        cust = customers_df.iloc[cust_idx]
        y = d.year
        
        status = order_status_for_year(y)
        approved, ship_dt, delivered_dt, estimated_dt = delivery_dates(pd.Timestamp(d) + pd.to_timedelta(np.random.randint(0, 24), unit="h"))
        if status == "canceled":
            delivered_dt = pd.NaT
        
        order_rows.append([
            order_id, cust["customer_id"], status,
            d, approved, ship_dt, delivered_dt, estimated_dt
        ])
        
        # items
        n_items = np.random.choice([1,2,3,4], p=[0.65,0.22,0.10,0.03])
        for item_no in range(1, n_items+1):
            pid, cat = sample_product()
            sid = sellers_df.sample(1, weights=seller_probs).iloc[0]["seller_id"]
            price = price_for_category(cat, d)
            freight = freight_from_price(price)
            ship_limit = approved + pd.to_timedelta(np.random.randint(6, 48), unit="h")
            item_rows.append([order_id, item_no, pid, sid, ship_limit, price, freight])
        
        # payments (1-2 sequential payments; installments common)
        payment_seqs = np.random.choice([1,2], p=[0.85,0.15])
        remaining_value = 0.0
        # total order value approximated as sum of item prices + freight
        # We'll calculate later by grouping to assign payment values realistically
        for seq in range(1, payment_seqs+1):
            ptype = np.random.choice(
                ["credit_card","boleto","voucher","debit_card"],
                p=[0.70,0.18,0.07,0.05]
            )
            installments = int(np.random.choice([1,2,3,6,10,12], p=[0.4,0.15,0.15,0.15,0.10,0.05]))
            # placeholder payment value; fix later based on order total
            payment_rows.append([order_id, seq, ptype, installments, np.nan])
        
        # reviews only for delivered
        if status == "delivered" and pd.notnull(delivered_dt):
            review_id_seq += 1
            delayed = (delivered_dt - estimated_dt) > pd.Timedelta(days=1)
            review_score = review_score_for_year(y, delayed=delayed)
            # review created near/after delivery
            review_created = delivered_dt + pd.to_timedelta(np.random.randint(0, 10), unit="D")
            review_answer = review_created + pd.to_timedelta(np.random.randint(0, 5), unit="D")
            review_rows.append([
                f"rev_{review_id_seq:08d}", order_id, review_score, "", "",
                review_created, review_answer
            ])

# Build DataFrames
orders_df = pd.DataFrame(order_rows, columns=[
    "order_id","customer_id","order_status","order_purchase_timestamp","order_approved_at",
    "order_delivered_carrier_date","order_delivered_customer_date","order_estimated_delivery_date"
])

items_df = pd.DataFrame(item_rows, columns=[
    "order_id","order_item_id","product_id","seller_id","shipping_limit_date","price","freight_value"
])

payments_df = pd.DataFrame(payment_rows, columns=[
    "order_id","payment_sequential","payment_type","payment_installments","payment_value"
])

reviews_df = pd.DataFrame(review_rows, columns=[
    "review_id","order_id","review_score","review_comment_title","review_comment_message",
    "review_creation_date","review_answer_timestamp"
])

# Assign realistic payment_value based on order totals
order_totals = items_df.groupby("order_id")[["price","freight_value"]].sum().sum(axis=1)
# For missing orders (canceled with no items, unlikely), handle safely
order_totals = order_totals.reindex(orders_df["order_id"]).fillna(0.0)

# Distribute order total across payment rows per order
def allocate_payments(df_pay):
    allocated = []
    for oid, grp in df_pay.groupby("order_id"):
        total = float(order_totals.get(oid, 0.0))
        if total <= 0:
            vals = [0.0]*len(grp)
        else:
            shares = np.random.dirichlet(np.ones(len(grp)))
            vals = list(np.round(shares * total, 2))
            # fix rounding
            diff = round(total - sum(vals), 2)
            if vals:
                vals[0] = round(vals[0] + diff, 2)
        g2 = grp.copy()
        g2["payment_value"] = vals
        allocated.append(g2)
    return pd.concat(allocated, ignore_index=True)

payments_df = allocate_payments(payments_df)

# -----------------------------
# 6) Save all CSVs
# -----------------------------

files = {
    "olist_customers_dataset.csv": customers_df,
    "olist_orders_dataset.csv": orders_df,
    "olist_order_items_dataset.csv": items_df,
    "olist_products_dataset.csv": products_df,
    "olist_sellers_dataset.csv": sellers_df,
    "olist_geolocation_dataset.csv": geolocation_df,
    "olist_order_payments_dataset.csv": payments_df,
    "olist_order_reviews_dataset.csv": reviews_df,
    "product_category_name_translation.csv": translation_df
}

for fname, df in files.items():
    df.to_csv(os.path.join(OUT_DIR, fname), index=False)

# Zip it
zip_path = "/mnt/data/olist_synthetic/olist_synthetic_2018_09_to_2023_12.zip"
with zipfile.ZipFile(zip_path, "w", compression=zipfile.ZIP_DEFLATED) as zf:
    for fname in files.keys():
        zf.write(os.path.join(OUT_DIR, fname), arcname=fname)

ModuleNotFoundError: No module named 'caas_jupyter_tools'

In [4]:
import os
import random
import string
import numpy as np
import pandas as pd
from faker import Faker
from datetime import datetime, timedelta

fake = Faker("pt_BR")
np.random.seed(42)
random.seed(42)

# ----------------------------
# Helper Functions
# ----------------------------
def rand_id(prefix, size=10):
    return prefix + "_" + "".join(random.choices(string.ascii_lowercase + string.digits, k=size))

def random_date(start, end):
    delta = end - start
    return start + timedelta(seconds=random.randint(0, int(delta.total_seconds())))

# ----------------------------
# Simulation parameters
# ----------------------------
start_date = datetime(2018, 9, 1)
end_date   = datetime(2023, 12, 31)

n_customers = 50000
n_sellers   = 2000
n_products  = 5000
n_orders    = 120000
n_geo       = 3000

out_dir = r"D:\Portfolio Projects\Ecommerce\Data\olist_synthetic"
os.makedirs(out_dir, exist_ok=True)

# ----------------------------
# Customers
# ----------------------------
customers = []
for i in range(n_customers):
    customers.append({
        "customer_id": rand_id("cust"),
        "customer_unique_id": rand_id("uniq"),
        "customer_zip_code_prefix": fake.postcode()[:5],
        "customer_city": fake.city(),
        "customer_state": fake.state_abbr()
    })
customers_df = pd.DataFrame(customers)
customers_df.to_csv(os.path.join(out_dir, "olist_customers_dataset.csv"), index=False)

# ----------------------------
# Sellers
# ----------------------------
sellers = []
for i in range(n_sellers):
    sellers.append({
        "seller_id": rand_id("sell"),
        "seller_zip_code_prefix": fake.postcode()[:5],
        "seller_city": fake.city(),
        "seller_state": fake.state_abbr()
    })
sellers_df = pd.DataFrame(sellers)
sellers_df.to_csv(os.path.join(out_dir, "olist_sellers_dataset.csv"), index=False)

# ----------------------------
# Products
# ----------------------------
categories = [
    "bed_bath_table", "health_beauty", "sports_leisure", "computers_accessories",
    "housewares", "watches_gifts", "telephony", "garden_tools",
    "fashion_bags_accessories", "toys", "baby", "pet_shop"
]
products = []
for i in range(n_products):
    products.append({
        "product_id": rand_id("prod"),
        "product_category_name": random.choice(categories),
        "product_name_lenght": random.randint(20, 60),
        "product_description_lenght": random.randint(100, 500),
        "product_photos_qty": random.randint(1, 4),
        "product_weight_g": random.randint(100, 10000),
        "product_length_cm": random.randint(10, 100),
        "product_height_cm": random.randint(5, 60),
        "product_width_cm": random.randint(5, 60)
    })
products_df = pd.DataFrame(products)
products_df.to_csv(os.path.join(out_dir, "olist_products_dataset.csv"), index=False)

# ----------------------------
# Orders + Items + Payments + Reviews
# ----------------------------
orders, items, payments, reviews = [], [], [], []

def adjust_for_economy(order_date, category):
    """Adjust demand and cancellation probability based on real-world events"""
    year = order_date.year
    month = order_date.month

    # Base probability of cancellation
    cancel_prob = 0.05

    # 2020 COVID impact
    if year == 2020 and month in [3,4,5]:
        if category in ["fashion_bags_accessories", "sports_leisure", "watches_gifts"]:
            return 0.5  # massive cancellation
        if category in ["bed_bath_table", "computers_accessories", "health_beauty"]:
            return 0.02  # strong demand, low cancellations
        cancel_prob = 0.15

    # Inflation in 2022: slightly higher cancellations
    if year == 2022:
        cancel_prob += 0.07

    return cancel_prob

for i in range(n_orders):
    order_id = rand_id("ord")
    cust = customers_df.sample(1).iloc[0]
    order_purchase = random_date(start_date, end_date)

    product = products_df.sample(1).iloc[0]
    cancel_prob = adjust_for_economy(order_purchase, product.product_category_name)

    order_status = "delivered" if random.random() > cancel_prob else "canceled"

    order_approved = order_purchase + timedelta(hours=random.randint(1, 72))
    order_delivered = order_approved + timedelta(days=random.randint(1, 15)) if order_status == "delivered" else None
    order_estimated = order_approved + timedelta(days=random.randint(5, 20))

    orders.append({
        "order_id": order_id,
        "customer_id": cust.customer_id,
        "order_status": order_status,
        "order_purchase_timestamp": order_purchase,
        "order_approved_at": order_approved,
        "order_delivered_carrier_date": order_approved + timedelta(days=1),
        "order_delivered_customer_date": order_delivered,
        "order_estimated_delivery_date": order_estimated
    })

    # Items
    n_items = random.randint(1, 3)
    for j in range(n_items):
        seller = sellers_df.sample(1).iloc[0]
        price = random.uniform(20, 500)
        freight = random.uniform(5, 50)
        items.append({
            "order_id": order_id,
            "order_item_id": j + 1,
            "product_id": product.product_id,
            "seller_id": seller.seller_id,
            "shipping_limit_date": order_approved + timedelta(days=2),
            "price": round(price, 2),
            "freight_value": round(freight, 2)
        })

    # Payments
    payments.append({
        "order_id": order_id,
        "payment_sequential": 1,
        "payment_type": random.choices(
            ["credit_card", "boleto", "voucher", "debit_card", "pix"],
            weights=[0.6, 0.15, 0.05, 0.1, 0.1]
        )[0],
        "payment_installments": random.randint(1, 12),
        "payment_value": round(price + freight, 2)
    })

    # Reviews
    if order_status == "delivered" and order_delivered:
        reviews.append({
            "review_id": rand_id("rev"),
            "order_id": order_id,
            "review_score": random.choices([1, 2, 3, 4, 5], weights=[0.1, 0.1, 0.2, 0.3, 0.3])[0],
            "review_comment_title": fake.sentence(nb_words=5),
            "review_comment_message": fake.text(max_nb_chars=200),
            "review_creation_date": order_delivered + timedelta(days=random.randint(0, 5)),
            "review_answer_timestamp": order_delivered + timedelta(days=random.randint(1, 7))
        })

# Save files
pd.DataFrame(orders).to_csv(os.path.join(out_dir, "olist_orders_dataset.csv"), index=False)
pd.DataFrame(items).to_csv(os.path.join(out_dir, "olist_order_items_dataset.csv"), index=False)
pd.DataFrame(payments).to_csv(os.path.join(out_dir, "olist_order_payments_dataset.csv"), index=False)
pd.DataFrame(reviews).to_csv(os.path.join(out_dir, "olist_order_reviews_dataset.csv"), index=False)

cat_translation = pd.DataFrame({
    "product_category_name": categories,
    "product_category_name_english": [c.replace("_", " ").title() for c in categories]
})
cat_translation.to_csv(os.path.join(out_dir, "product_category_name_translation.csv"), index=False)

print(f"Synthetic event-aware Olist dataset generated in: {out_dir}")


Synthetic event-aware Olist dataset generated in: D:\Portfolio Projects\Ecommerce\Data\olist_synthetic


In [3]:
pip install faker


                                              0.0/1.9 MB ? eta -:--:--
                                              0.0/1.9 MB 1.4 MB/s eta 0:00:02
                                              0.0/1.9 MB 495.5 kB/s eta 0:00:04
     -                                        0.1/1.9 MB 656.4 kB/s eta 0:00:03
     --                                       0.1/1.9 MB 656.4 kB/s eta 0:00:03
     --                                       0.1/1.9 MB 656.4 kB/s eta 0:00:03
     --                                       0.1/1.9 MB 656.4 kB/s eta 0:00:03
     --                                       0.1/1.9 MB 426.7 kB/s eta 0:00:05
     ---                                      0.2/1.9 MB 499.5 kB/s eta 0:00:04
     ---                                      0.2/1.9 MB 472.6 kB/s eta 0:00:04
     -----                                    0.3/1.9 MB 610.3 kB/s eta 0:00:03
     -------                                  0.4/1.9 MB 696.3 kB/s eta 0:00:03
     --------                                 0.4/1